In [ ]:
db=[["harsh",1111,"ok"],
     ["abhi",2222,"good"],
   ["rahul",3333,"ok"]]

In [ ]:
db

In [ ]:
db[ : ,1]

In [ ]:

print((db[0][1], db[1][1], db[2][1]))

In [1]:
import cv2
import pandas as pd
from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
import datetime
import os
import time

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize detectors
try:
    face_detector = FaceDetector(minDetectionCon=0.5)
    use_cvzone_face = True
except:
    print("CVZone Face Detection failed, using Haar Cascade instead")
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    use_cvzone_face = False

hand_detector = HandDetector(detectionCon=0.8, maxHands=1)

# Get user name
user_name = input("Enter your name for attendance: ")

# Initialize attendance log
attendance_file = 'attendance.xlsx'
if not os.path.exists(attendance_file):
    df = pd.DataFrame(columns=["Name", "Date", "Time", "Status"])
    df.to_excel(attendance_file, index=False)
else:
    df = pd.read_excel(attendance_file)


print("\n=== Attendance System Ready ===")
print("1. Show your face clearly")
print("2. Raise one finger to mark attendance")

attendance_marked = False

while True:
    success, img = cap.read()
    if not success or attendance_marked:
        break

    # Detect faces
    if use_cvzone_face:
        img, faces = face_detector.findFaces(img)
        face_bboxes = [face["bbox"] for face in faces]
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        face_bboxes = faces
    
    # Detect hands
    hands, img = hand_detector.findHands(img)
    
    # Check conditions for attendance
    if len(face_bboxes) > 0 and hands:
        bbox = face_bboxes[0]
        x, y, w, h = bbox if use_cvzone_face else (bbox[0], bbox[1], bbox[2], bbox[3])
        
        hand = hands[0]
        fingers = hand_detector.fingersUp(hand)
        
        if sum(fingers) == 1:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            current_time = datetime.datetime.now()
            date_str = current_time.strftime("%Y-%m-%d")
            time_str = current_time.strftime("%H:%M:%S")
            
            # Record attendance
            new_entry = pd.DataFrame({
                "Name": [user_name],
                "Date": [date_str],
                "Time": [time_str],
                "Status": ["Present"]
            })
            
            # Update Excel file
            updated_data = pd.concat([df, new_entry], ignore_index=True)
            updated_data.to_excel(attendance_file, index=False)
            
            status_text = f"Attendance recorded for {user_name}"
            print(f"{time_str} - Attendance marked for {user_name}")
            
            cv2.putText(img, status_text, (x, y-20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.imshow("Attendance System", img)
            cv2.waitKey(2000)  # Show confirmation for 2 seconds
            attendance_marked = True
            break
    
    # Display instructions
    cv2.putText(img, f"User: {user_name}", (10, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(img, "Show face and raise one finger", (10, 60), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(img, "to mark attendance", (10, 90), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    cv2.imshow("Attendance System", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Enter your name for attendance:  Deek



=== Attendance System Ready ===
1. Show your face clearly
2. Raise one finger to mark attendance
11:15:26 - Attendance marked for Deek
